In [1]:
import arcpy
import requests
import json
import pprint
import zipfile
import arcpy
import pandas as pd
import os
import numpy as np
from datetime import timedelta
from datetime import datetime

In [5]:
    # Run Cross Validation to get results
    cvResult = arcpy.ga.CrossValidation("test", r"C:\Users\ecava\Documents\ArcGIS\Projects\Lab2\Lab2.gdb\testcv")
    # Add RMSE to a list
    print(cvResult.rootMeanSquare)

4.481785432784913


# Request Station Data

In [9]:
# Adding my token to a variable
Token = "EWfeohdvgAYBUAOZMoPgKNBjgaFAjMXS"

In [10]:
# Writing my params for which stations to choose
station_params = {'datasetid': 'GHCND', 'startdate': (datetime.now() - timedelta(days=365)).strftime('%Y-%m-%d'),
               'enddate': datetime.now().strftime('%Y-%m-%d'), 'locationid': 'FIPS:27', 'datatypeid': ['TMIN', 'TMAX'], 'limit':'1000'}

In [11]:
# Adding the base url to a variable - Getting Stations
base_url = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/stations'

In [12]:
# Making the request for specific stations
stations_req = requests.get(base_url, params=station_params, headers={'token':Token})
stations_req

<Response [200]>

In [13]:
# Translating the request into JSON format and printing
stations_j = stations_req.json()

In [14]:
stations_j

{'metadata': {'resultset': {'offset': 1, 'count': 178, 'limit': 1000}},
 'results': [{'elevation': 335,
   'mindate': '1978-12-01',
   'maxdate': '2022-04-30',
   'latitude': 48.6333,
   'name': 'BARWICK, MN US',
   'datacoverage': 0.9529,
   'id': 'GHCND:CA006020559',
   'elevationUnit': 'METERS',
   'longitude': -93.9667},
  {'elevation': 276.5,
   'mindate': '1893-01-01',
   'maxdate': '2022-04-24',
   'latitude': 47.2991,
   'name': 'ADA, MN US',
   'datacoverage': 0.863,
   'id': 'GHCND:USC00210018',
   'elevationUnit': 'METERS',
   'longitude': -96.5161},
  {'elevation': 348.1,
   'mindate': '1957-04-01',
   'maxdate': '2022-02-28',
   'latitude': 48.3005,
   'name': 'AGASSIZ REFUGE, MN US',
   'datacoverage': 0.8196,
   'id': 'GHCND:USC00210050',
   'elevationUnit': 'METERS',
   'longitude': -95.9816},
  {'elevation': 370.3,
   'mindate': '1940-06-01',
   'maxdate': '2021-12-31',
   'latitude': 46.5257,
   'name': 'AITKIN 2 E, MN US',
   'datacoverage': 0.8745,
   'id': 'GHCND:U

# Creating a Point Feature Class for the Available Stations

In [ ]:
# Creating an empty feature class of type Point
stations = stations_j['results']
workspace = arcpy.env.workspace
arcpy.CreateFeatureclass_management(workspace,"stations","POINT","","DISABLED","DISABLED",arcpy.SpatialReference(4269))

In [ ]:
# Adding fields to the feature class
arcpy.AddField_management("stations","id","TEXT","","",100)
arcpy.AddField_management("stations","name","TEXT","","",100)
arcpy.AddField_management("stations","mindate","DATE")
arcpy.AddField_management("stations","maxdate","DATE")

In [ ]:
# Creating a cursor to add the info to the feature class
cur = arcpy.da.InsertCursor("stations", ["id", "name", "mindate", "maxdate", "Shape@"])

for s in stations:
   
    point = arcpy.Point(s['longitude'],s['latitude'])
    row = [s["id"],s["name"],s["mindate"],s["maxdate"],point]
   
    cur.insertRow(row)
   
del cur

# Request Temperature Data

In [18]:
# Creating a list of the stations that can be looped through to download data
stations_list = []
for station in stations:
    stations_list.append(station['id'])

In [19]:
# Base URL for requesting temperature data
data_req_url = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?'

## Daily Minimum Temps at Each Station

In [20]:
# Creating a dictionary of the stations to hold Min temp data for each station
stations_dict_min = dict.fromkeys(stations_list)

In [21]:
# Looping through the list of stations 
# and making requests for the min temp for each station
# for the time period.
for s_name in stations_list:
    min_params = {'datasetid': 'GHCND', 'datatypeid': 'TMIN', 'units': 'standard', 'limit': '365', 'stationid': s_name, 'startdate': (datetime.now() - timedelta(days=365)).strftime('%Y-%m-%d'), 'enddate': datetime.now().strftime('%Y-%m-%d')}
    # make the api call
    min_req = requests.get(data_req_url, params = min_params, headers = {'token':Token})
    # load the api response as a json
    min_d = min_req.json()
    stations_dict_min[s_name] = min_d

In [23]:
# Creating an Array of Arrays by looping through
# The min-temp json and grabbing the date, temp, and station id and
# Appending them to a list then appending that to the 'all-station array'
min_all_station_array = []
for station in stations_dict_min:
    for results in stations_dict_min[station]['results']:
        station_array = []
        station_array.append(results['date'])
        station_array.append(results['value'])
        station_array.append(results['station'])
        min_all_station_array.append(station_array)

In [24]:
# Creating a dataframe from the array and naming the columns
all_min_temps = pd.DataFrame(min_all_station_array, columns = ['datetime', 'min_temp', 'station'])

In [ ]:
# Pivoting the dataframe so that the index is station id and each day is a column
min_station_data = df.pivot(index = 'station', columns = 'datetime', values = 'min_temp')
min_station_data

## Daily Maximum Temps at Each Station

In [ ]:
# Creating a dictionary of the stations to hold data for each station
stations_dict_max = dict.fromkeys(stations_list)

In [ ]:
# Looping through the list of stations and making requests for the max temp for each station
# for the time period.
for s_name in stations_list:
    max_params = {'datasetid': 'GHCND', 'datatypeid': 'TMAX', 'units': 'standard', 'limit': '365', 'stationid': s_name, 'startdate': (datetime.now() - timedelta(days=365)).strftime('%Y-%m-%d'), 'enddate': datetime.now().strftime('%Y-%m-%d')}
    # make the api call
    max_req = requests.get(data_req_url, params = max_params, headers = {'token':Token})
    # load the api response as a json
    max_d = max_req.json()
    stations_dict_max[s_name] = max_d

In [ ]:
all_station_array = []
for station in stations_dict_max:
    for results in stations_dict_max[station]['results']:
        station_array = []
        station_array.append(results['date'])
        station_array.append(results['value'])
        station_array.append(results['station'])
        all_station_array.append(station_array)

In [ ]:
df_max = pd.DataFrame(all_station_array, columns = ['datetime', 'max_temp', 'station'])

In [ ]:
# Pivoting the dataframe so that the index is station id and each day is a column
df_max_pivot_station = df_max.pivot(index = 'station', columns = 'datetime', values = 'max_temp')
df_max_pivot_station

## Joining both min and max to one df > csv

In [ ]:
all_data = pd.merge(df_pivot_station, df_max_pivot_station, 'outer', on = 'station')

In [ ]:
all_data.to_csv(r"C:\Users\ecava\Documents\ArcGIS\Projects\Lab2\temp_data.csv")

In [ ]:
# Join
arcpy.management.AddJoin("stations", "id", "temp_data.csv", "station", "KEEP_ALL", "NO_INDEX_JOIN_FIELDS")

In [ ]:
# Arcpy describe or pd columns
days_list = list(all_data.columns)

# QCQA here after

In [2]:
import pandas as pd
temp_dat = r'C:\Users\umn-ahmad178\Desktop\lab3\final2\temp_data.csv'
temp_pd = pd.read_csv(temp_dat)

temp_pd

,station,2021-05-02 00:00:00_x,2021-05-03 00:00:00_x,2021-05-04 00:00:00_x,2021-05-05 00:00:00_x,2021-05-06 00:00:00_x,2021-05-07 00:00:00_x,2021-05-08 00:00:00_x,2021-05-09 00:00:00_x,2021-05-10 00:00:00_x,...,2022-04-18 00:00:00_y,2022-04-19 00:00:00_y,2022-04-20 00:00:00_y,2022-04-21 00:00:00_y,2022-04-22 00:00:00_y,2022-04-23 00:00:00_y,2022-04-24 00:00:00_y,2022-04-25 00:00:00_y,2022-04-26 00:00:00_y,2022-04-27 00:00:00_y
0,GHCND:CA006020559,35.0,43.0,36.0,24.0,23.0,28.0,25.0,28.0,30.0,...,36.0,43.0,36.0,41.0,41.0,48.0,45.0,42.0,36.0,NaN
1,GHCND:USC00210018,44.0,39.0,25.0,24.0,25.0,23.0,25.0,25.0,27.0,...,NaN,NaN,NaN,44.0,42.0,43.0,72.0,NaN,NaN,NaN
2,GHCND:USC00210050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,GHCND:USC00210059,38.0,36.0,38.0,27.0,29.0,28.0,30.0,34.0,30.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,GHCND:USC00210075,58.0,46.0,40.0,37.0,39.0,41.0,38.0,38.0,41.0,...,52.0,44.0,51.0,54.0,61.0,67.0,73.0,57.0,50.0,56.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,GHCND:USW00094960,50.0,47.0,37.0,33.0,40.0,34.0,33.0,36.0,33.0,...,38.0,48.0,46.0,56.0,55.0,75.0,53.0,39.0,43.0,50.0
174,GHCND:USW00094961,34.0,39.0,34.0,25.0,25.0,30.0,28.0,33.0,30.0,...,37.0,46.0,40.0,44.0,43.0,50.0,51.0,30.0,38.0,48.0
175,GHCND:USW00094963,54.0,48.0,41.0,37.0,37.0,38.0,37.0,37.0,40.0,...,37.0,47.0,46.0,56.0,56.0,75.0,50.0,37.0,43.0,51.0
176,GHCND:USW00094967,49.0,41.0,30.0,28.0,29.0,29.0,25.0,29.0,30.0,...,34.0,43.0,39.0,45.0,43.0,68.0,47.0,30.0,35.0,48.0


In [3]:
# temp_pd.transpose()
temp_pd_tp = temp_pd.transpose().iloc[1:].rename(columns=temp_pd.transpose().iloc[0]).reset_index()
temp_pd_tp

,index,GHCND:CA006020559,GHCND:USC00210018,GHCND:USC00210050,GHCND:USC00210059,GHCND:USC00210075,GHCND:USC00210190,GHCND:USC00210287,GHCND:USC00210355,GHCND:USC00210387,...,GHCND:USW00014927,GHCND:USW00014992,GHCND:USW00054932,GHCND:USW00094931,GHCND:USW00094938,GHCND:USW00094960,GHCND:USW00094961,GHCND:USW00094963,GHCND:USW00094967,GHCND:USW00094992
0,2021-05-02 00:00:00_x,35,44,NaN,38,58,49,53,46,41,...,51,52,42,43,50,50,34,54,49,39
1,2021-05-03 00:00:00_x,43,39,NaN,36,46,45,44,55,38,...,50,43,40,38,43,47,39,48,41,40
2,2021-05-04 00:00:00_x,36,25,NaN,38,40,37,30,39,35,...,41,36,33,27,32,37,34,41,30,39
3,2021-05-05 00:00:00_x,24,24,NaN,27,37,29,35,34,25,...,35,33,28,22,28,33,25,37,28,34
4,2021-05-06 00:00:00_x,23,25,NaN,29,39,30,35,38,23,...,38,34,31,23,35,40,25,37,29,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717,2022-04-23 00:00:00_y,48,43,NaN,NaN,67,NaN,77,62,NaN,...,75,79,70,51,74,75,50,75,68,47
718,2022-04-24 00:00:00_y,45,72,NaN,NaN,73,NaN,59,73,NaN,...,53,48,56,49,54,53,51,50,47,46
719,2022-04-25 00:00:00_y,42,NaN,NaN,NaN,57,NaN,37,47,NaN,...,38,34,35,33,34,39,30,37,30,40
720,2022-04-26 00:00:00_y,36,NaN,NaN,NaN,50,NaN,38,35,NaN,...,43,43,36,31,36,43,38,43,35,34


In [5]:
original_null_percent = list((temp_pd_tp.isna().sum()/len(temp_pd_tp))*100)
original_null_percent

[0.0,
 0.13850415512465375,
 11.911357340720222,
 23.130193905817176,
 32.686980609418285,
 0.2770083102493075,
 16.06648199445983,
 2.7700831024930745,
 1.3850415512465373,
 31.57894736842105,
 25.90027700831025,
 4.570637119113574,
 31.440443213296398,
 0.41551246537396125,
 11.634349030470915,
 21.88365650969529,
 0.0,
 7.894736842105263,
 9.141274238227147,
 0.8310249307479225,
 4.847645429362881,
 12.326869806094184,
 4.986149584487535,
 0.6925207756232686,
 4.1551246537396125,
 0.6925207756232686,
 11.218836565096952,
 0.0,
 32.40997229916898,
 28.80886426592798,
 11.634349030470915,
 19.94459833795014,
 33.933518005540165,
 92.797783933518,
 8.033240997229916,
 0.554016620498615,
 0.13850415512465375,
 5.8171745152354575,
 7.756232686980609,
 0.0,
 31.024930747922436,
 4.293628808864266,
 1.3850415512465373,
 22.160664819944596,
 23.822714681440445,
 5.540166204986149,
 4.293628808864266,
 13.71191135734072,
 12.742382271468145,
 20.3601108033241,
 32.40997229916898,
 19.1135734

In [4]:
# download temperature data using the meteostat module (since 2022 March 1)
import arcpy
import pandas as pd
import arcgisscripting
from datetime import date, timedelta
from datetime import datetime
from meteostat import Stations, Daily, Point

gp = arcgisscripting.create()
path = r'C:\Users\umn-ahmad178\Desktop\lab3\final2'
shp = r'C:\Users\umn-ahmad178\Desktop\lab3\final2\dailytemp_bystation\dailytemp_bystation.shp'
arcpy.env.workspace = path

def get_weather(start, end, lat, lon, station_id, station_n):
    start = start.split('-')
    end = end.split('-')
    start = datetime(int(start[0]), int(start[1]), int(start[2]))
    end = datetime(int(end[0]), int(end[1]), int(end[2]))
    location = Point(lat, lon)
    met_data = Daily(location, start, end)
    met_data = met_data.fetch()
    met_data['station_id'] = station_id
    met_data['station_n'] = station_n
    met_data['lat'] = lat
    met_data['lon'] = lon
    return met_data

meteostat_data = pd.DataFrame([])
today = datetime.today().strftime('%Y-%m-%d')
yesterday = (date.today() - timedelta(days=2)).strftime('%Y-%m-%d')

#loop through the shapefile of the mn counties
with arcpy.da.SearchCursor(shp, ["SHAPE@XY", 'stations_i', 'stations_n']) as cursor:
    for row in cursor:
        station_id = str(row[1])
        station_n = str(row[2])
        lon = float(row[0][0])
        lat = float(row[0][1])
        temp_data = get_weather('2021-05-02', '2022-04-27', lat, lon, station_id, station_n).reset_index()
        meteostat_data = pd.concat([meteostat_data, temp_data])
    
meteostat_data.reset_index(drop=True, inplace=True)
meteostat_data = meteostat_data[['station_id','station_n','lat','lon','time','tavg','tmin','tmax']]
meteostat_data

    

,station_id,station_n,lat,lon,time,tavg,tmin,tmax
0,GHCND:CA006020559,"BARWICK, MN US",48.633300,-93.966700,2021-05-02,11.8,1.5,22.0
1,GHCND:CA006020559,"BARWICK, MN US",48.633300,-93.966700,2021-05-03,7.5,6.0,9.0
2,GHCND:CA006020559,"BARWICK, MN US",48.633300,-93.966700,2021-05-04,6.0,2.0,10.0
3,GHCND:CA006020559,"BARWICK, MN US",48.633300,-93.966700,2021-05-05,3.5,-4.5,11.5
4,GHCND:CA006020559,"BARWICK, MN US",48.633300,-93.966700,2021-05-06,3.8,-5.0,12.5
...,...,...,...,...,...,...,...,...
56688,GHCND:USW00094992,"GRAND MARAIS, MN US",47.745404,-90.345639,2022-04-23,4.5,2.4,8.0
56689,GHCND:USW00094992,"GRAND MARAIS, MN US",47.745404,-90.345639,2022-04-24,4.4,2.0,7.0
56690,GHCND:USW00094992,"GRAND MARAIS, MN US",47.745404,-90.345639,2022-04-25,1.8,-2.0,4.0
56691,GHCND:USW00094992,"GRAND MARAIS, MN US",47.745404,-90.345639,2022-04-26,-2.0,-4.0,1.0


In [6]:
# Replace NaN values with available data from meteostat module
count_min = 0
count_max = 0
count_all = 0
for i in range(1, len(list(temp_pd_tp.columns))):
    for j in range(len(temp_pd_tp.iloc[:,i].isna())):
        if temp_pd_tp.iloc[:,i].isna()[j]:
            count_all += 1
            st_id = temp_pd_tp.iloc[:,i].name
            date = temp_pd_tp.loc[j,'index'][0:10]
            if temp_pd_tp.loc[j,'index'].endswith('x'):
                new_min = list(meteostat_data[(meteostat_data['time']==date)&(meteostat_data['station_id']==st_id)]['tmin'])
                if len(new_min) != 0:
                    count_min += 1
                    temp_pd_tp.iloc[:,i][j] = new_min[0]
            elif temp_pd_tp.loc[j,'index'].endswith('y'):
                new_max = list(meteostat_data[(meteostat_data['time']==date)&(meteostat_data['station_id']==st_id)]['tmax'])
                if len(new_max) != 0:
                    count_max += 1
                    temp_pd_tp.iloc[:,i][j] = new_max[0]
                    
print(f'from a total of {count_all} values, {count_min} min values, and {count_max} max values were replaced.')                  

from a total of 11455 values, 4547 min values, and 4461 max values were replaced.


In [7]:
new_null_percent = list((temp_pd_tp.isna().sum()/len(temp_pd_tp))*100)
new_null_percent

[0.0,
 0.0,
 11.911357340720222,
 22.022160664819946,
 0.2770083102493075,
 0.0,
 0.2770083102493075,
 0.0,
 0.0,
 0.2770083102493075,
 0.2770083102493075,
 0.0,
 31.440443213296398,
 0.0,
 0.0,
 0.2770083102493075,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 12.326869806094184,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.2770083102493075,
 0.0,
 0.2770083102493075,
 0.0,
 0.13850415512465375,
 0.2770083102493075,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.2770083102493075,
 0.0,
 1.3850415512465373,
 0.2770083102493075,
 0.2770083102493075,
 0.0,
 0.0,
 0.2770083102493075,
 0.0,
 20.3601108033241,
 0.2770083102493075,
 0.2770083102493075,
 0.2770083102493075,
 10.249307479224377,
 0.0,
 0.0,
 0.0,
 14.40443213296399,
 0.2770083102493075,
 0.0,
 92.5207756232687,
 0.0,
 0.0,
 0.0,
 0.2770083102493075,
 0.0,
 0.2770083102493075,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.2770083102493075,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.2770083102493075,
 0.0,
 0.0,
 13.573407202216067,
 0.0,
 0.2770083102493075,
 0.27700

In [10]:
# the percent of the records that were filled
diffs = [original_null_percent[i]-new_null_percent[i] for i in range(len(original_null_percent))]
diffs

[0.0,
 0.13850415512465375,
 0.0,
 1.10803324099723,
 32.40997229916898,
 0.2770083102493075,
 15.789473684210524,
 2.7700831024930745,
 1.3850415512465373,
 31.301939058171744,
 25.623268698060944,
 4.570637119113574,
 0.0,
 0.41551246537396125,
 11.634349030470915,
 21.60664819944598,
 0.0,
 7.894736842105263,
 9.141274238227147,
 0.8310249307479225,
 4.847645429362881,
 0.0,
 4.986149584487535,
 0.6925207756232686,
 4.1551246537396125,
 0.6925207756232686,
 11.218836565096952,
 0.0,
 32.13296398891967,
 28.80886426592798,
 11.357340720221607,
 19.94459833795014,
 33.795013850415515,
 92.52077562326869,
 8.033240997229916,
 0.554016620498615,
 0.13850415512465375,
 5.8171745152354575,
 7.756232686980609,
 0.0,
 30.74792243767313,
 4.293628808864266,
 0.0,
 21.88365650969529,
 23.545706371191137,
 5.540166204986149,
 4.293628808864266,
 13.434903047091412,
 12.742382271468145,
 0.0,
 32.13296398891967,
 18.83656509695291,
 22.71468144044321,
 0.0,
 25.346260387811636,
 0.1385041551246

In [13]:
new_null_percent

[0.0,
 0.0,
 11.911357340720222,
 22.022160664819946,
 0.2770083102493075,
 0.0,
 0.2770083102493075,
 0.0,
 0.0,
 0.2770083102493075,
 0.2770083102493075,
 0.0,
 31.440443213296398,
 0.0,
 0.0,
 0.2770083102493075,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 12.326869806094184,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.2770083102493075,
 0.0,
 0.2770083102493075,
 0.0,
 0.13850415512465375,
 0.2770083102493075,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.2770083102493075,
 0.0,
 1.3850415512465373,
 0.2770083102493075,
 0.2770083102493075,
 0.0,
 0.0,
 0.2770083102493075,
 0.0,
 20.3601108033241,
 0.2770083102493075,
 0.2770083102493075,
 0.2770083102493075,
 10.249307479224377,
 0.0,
 0.0,
 0.0,
 14.40443213296399,
 0.2770083102493075,
 0.0,
 92.5207756232687,
 0.0,
 0.0,
 0.0,
 0.2770083102493075,
 0.0,
 0.2770083102493075,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.2770083102493075,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.2770083102493075,
 0.0,
 0.0,
 13.573407202216067,
 0.0,
 0.2770083102493075,
 0.27700

In [17]:
temp_pd_tp

,index,GHCND:CA006020559,GHCND:USC00210018,GHCND:USC00210050,GHCND:USC00210059,GHCND:USC00210075,GHCND:USC00210190,GHCND:USC00210287,GHCND:USC00210355,GHCND:USC00210387,...,GHCND:USW00014927,GHCND:USW00014992,GHCND:USW00054932,GHCND:USW00094931,GHCND:USW00094938,GHCND:USW00094960,GHCND:USW00094961,GHCND:USW00094963,GHCND:USW00094967,GHCND:USW00094992
0,2021-05-02 00:00:00_x,35,44,NaN,38,58,49,53,46,41,...,51,52,42,43,50,50,34,54,49,39
1,2021-05-03 00:00:00_x,43,39,NaN,36,46,45,44,55,38,...,50,43,40,38,43,47,39,48,41,40
2,2021-05-04 00:00:00_x,36,25,NaN,38,40,37,30,39,35,...,41,36,33,27,32,37,34,41,30,39
3,2021-05-05 00:00:00_x,24,24,NaN,27,37,29,35,34,25,...,35,33,28,22,28,33,25,37,28,34
4,2021-05-06 00:00:00_x,23,25,NaN,29,39,30,35,38,23,...,38,34,31,23,35,40,25,37,29,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717,2022-04-23 00:00:00_y,48,43,NaN,19,67,23,77,62,9,...,75,79,70,51,74,75,50,75,68,47
718,2022-04-24 00:00:00_y,45,72,9,13,73,13,59,73,8,...,53,48,56,49,54,53,51,50,47,46
719,2022-04-25 00:00:00_y,42,NaN,-3,0,57,4,37,47,2,...,38,34,35,33,34,39,30,37,30,40
720,2022-04-26 00:00:00_y,36,NaN,3,1,50,6,38,35,-1,...,43,43,36,31,36,43,38,43,35,34


# Growing Degree Days

For corn, the degree-day accumulation base is 50.

In [21]:
temp_mins = temp_pd_tp[temp_pd_tp['index'].str.contains('x')]
temp_maxes = temp_pd_tp[temp_pd_tp['index'].str.contains('y')]
temp_maxes

,index,GHCND:CA006020559,GHCND:USC00210018,GHCND:USC00210050,GHCND:USC00210059,GHCND:USC00210075,GHCND:USC00210190,GHCND:USC00210287,GHCND:USC00210355,GHCND:USC00210387,...,GHCND:USW00014927,GHCND:USW00014992,GHCND:USW00054932,GHCND:USW00094931,GHCND:USW00094938,GHCND:USW00094960,GHCND:USW00094961,GHCND:USW00094963,GHCND:USW00094967,GHCND:USW00094992
361,2021-05-02 00:00:00_y,72,86,NaN,89,91,88,87,91,77,...,68,71,70,75,73,67,70,70,75,53
362,2021-05-03 00:00:00_y,48,72,NaN,66,83,65,64,82,73,...,68,64,60,53,59,68,50,67,58,50
363,2021-05-04 00:00:00_y,50,52,NaN,53,67,65,58,67,47,...,61,60,57,52,56,60,51,59,54,58
364,2021-05-05 00:00:00_y,53,59,NaN,54,59,58,56,59,50,...,55,51,58,53,58,54,52,54,57,48
365,2021-05-06 00:00:00_y,54,57,NaN,60,57,56,68,58,51,...,63,65,60,55,61,62,56,62,62,57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717,2022-04-23 00:00:00_y,48,43,NaN,19,67,23,77,62,9,...,75,79,70,51,74,75,50,75,68,47
718,2022-04-24 00:00:00_y,45,72,9,13,73,13,59,73,8,...,53,48,56,49,54,53,51,50,47,46
719,2022-04-25 00:00:00_y,42,NaN,-3,0,57,4,37,47,2,...,38,34,35,33,34,39,30,37,30,40
720,2022-04-26 00:00:00_y,36,NaN,3,1,50,6,38,35,-1,...,43,43,36,31,36,43,38,43,35,34


In [22]:
temp_means = temp_maxes.copy()
temp_means


,index,GHCND:CA006020559,GHCND:USC00210018,GHCND:USC00210050,GHCND:USC00210059,GHCND:USC00210075,GHCND:USC00210190,GHCND:USC00210287,GHCND:USC00210355,GHCND:USC00210387,...,GHCND:USW00014927,GHCND:USW00014992,GHCND:USW00054932,GHCND:USW00094931,GHCND:USW00094938,GHCND:USW00094960,GHCND:USW00094961,GHCND:USW00094963,GHCND:USW00094967,GHCND:USW00094992
361,2021-05-02 00:00:00_y,72,86,NaN,89,91,88,87,91,77,...,68,71,70,75,73,67,70,70,75,53
362,2021-05-03 00:00:00_y,48,72,NaN,66,83,65,64,82,73,...,68,64,60,53,59,68,50,67,58,50
363,2021-05-04 00:00:00_y,50,52,NaN,53,67,65,58,67,47,...,61,60,57,52,56,60,51,59,54,58
364,2021-05-05 00:00:00_y,53,59,NaN,54,59,58,56,59,50,...,55,51,58,53,58,54,52,54,57,48
365,2021-05-06 00:00:00_y,54,57,NaN,60,57,56,68,58,51,...,63,65,60,55,61,62,56,62,62,57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
717,2022-04-23 00:00:00_y,48,43,NaN,19,67,23,77,62,9,...,75,79,70,51,74,75,50,75,68,47
718,2022-04-24 00:00:00_y,45,72,9,13,73,13,59,73,8,...,53,48,56,49,54,53,51,50,47,46
719,2022-04-25 00:00:00_y,42,NaN,-3,0,57,4,37,47,2,...,38,34,35,33,34,39,30,37,30,40
720,2022-04-26 00:00:00_y,36,NaN,3,1,50,6,38,35,-1,...,43,43,36,31,36,43,38,43,35,34


In [35]:
mean_dates = [el[:-1]+'m' for el in temp_means['index']]
mean_dates


['2021-05-02 00:00:00_m',
 '2021-05-03 00:00:00_m',
 '2021-05-04 00:00:00_m',
 '2021-05-05 00:00:00_m',
 '2021-05-06 00:00:00_m',
 '2021-05-07 00:00:00_m',
 '2021-05-08 00:00:00_m',
 '2021-05-09 00:00:00_m',
 '2021-05-10 00:00:00_m',
 '2021-05-11 00:00:00_m',
 '2021-05-12 00:00:00_m',
 '2021-05-13 00:00:00_m',
 '2021-05-14 00:00:00_m',
 '2021-05-15 00:00:00_m',
 '2021-05-16 00:00:00_m',
 '2021-05-17 00:00:00_m',
 '2021-05-18 00:00:00_m',
 '2021-05-19 00:00:00_m',
 '2021-05-20 00:00:00_m',
 '2021-05-21 00:00:00_m',
 '2021-05-22 00:00:00_m',
 '2021-05-23 00:00:00_m',
 '2021-05-24 00:00:00_m',
 '2021-05-25 00:00:00_m',
 '2021-05-26 00:00:00_m',
 '2021-05-27 00:00:00_m',
 '2021-05-28 00:00:00_m',
 '2021-05-29 00:00:00_m',
 '2021-05-30 00:00:00_m',
 '2021-05-31 00:00:00_m',
 '2021-06-01 00:00:00_m',
 '2021-06-02 00:00:00_m',
 '2021-06-03 00:00:00_m',
 '2021-06-04 00:00:00_m',
 '2021-06-05 00:00:00_m',
 '2021-06-06 00:00:00_m',
 '2021-06-07 00:00:00_m',
 '2021-06-08 00:00:00_m',
 '2021-06-09

In [82]:
gdd_dates = [el[:-1]+'g' for el in temp_means['index']]
# gdd_dates

temp_gdd = temp_maxes.copy()
# temp_gdd

In [83]:
temp_gdd['index'] = gdd_dates
temp_gdd = temp_gdd.reset_index(drop=True)
temp_gdd

,index,GHCND:CA006020559,GHCND:USC00210018,GHCND:USC00210050,GHCND:USC00210059,GHCND:USC00210075,GHCND:USC00210190,GHCND:USC00210287,GHCND:USC00210355,GHCND:USC00210387,...,GHCND:USW00014927,GHCND:USW00014992,GHCND:USW00054932,GHCND:USW00094931,GHCND:USW00094938,GHCND:USW00094960,GHCND:USW00094961,GHCND:USW00094963,GHCND:USW00094967,GHCND:USW00094992
0,2021-05-02 00:00:00_g,72,86,NaN,89,91,88,87,91,77,...,68,71,70,75,73,67,70,70,75,53
1,2021-05-03 00:00:00_g,48,72,NaN,66,83,65,64,82,73,...,68,64,60,53,59,68,50,67,58,50
2,2021-05-04 00:00:00_g,50,52,NaN,53,67,65,58,67,47,...,61,60,57,52,56,60,51,59,54,58
3,2021-05-05 00:00:00_g,53,59,NaN,54,59,58,56,59,50,...,55,51,58,53,58,54,52,54,57,48
4,2021-05-06 00:00:00_g,54,57,NaN,60,57,56,68,58,51,...,63,65,60,55,61,62,56,62,62,57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,2022-04-23 00:00:00_g,48,43,NaN,19,67,23,77,62,9,...,75,79,70,51,74,75,50,75,68,47
357,2022-04-24 00:00:00_g,45,72,9,13,73,13,59,73,8,...,53,48,56,49,54,53,51,50,47,46
358,2022-04-25 00:00:00_g,42,NaN,-3,0,57,4,37,47,2,...,38,34,35,33,34,39,30,37,30,40
359,2022-04-26 00:00:00_g,36,NaN,3,1,50,6,38,35,-1,...,43,43,36,31,36,43,38,43,35,34


In [49]:
# making a template
temp_means['index'] = mean_dates
temp_means = temp_means.reset_index(drop=True)
temp_means

,index,GHCND:CA006020559,GHCND:USC00210018,GHCND:USC00210050,GHCND:USC00210059,GHCND:USC00210075,GHCND:USC00210190,GHCND:USC00210287,GHCND:USC00210355,GHCND:USC00210387,...,GHCND:USW00014927,GHCND:USW00014992,GHCND:USW00054932,GHCND:USW00094931,GHCND:USW00094938,GHCND:USW00094960,GHCND:USW00094961,GHCND:USW00094963,GHCND:USW00094967,GHCND:USW00094992
0,2021-05-02 00:00:00_m,72,86,NaN,89,91,88,87,91,77,...,68,71,70,75,73,67,70,70,75,53
1,2021-05-03 00:00:00_m,48,72,NaN,66,83,65,64,82,73,...,68,64,60,53,59,68,50,67,58,50
2,2021-05-04 00:00:00_m,50,52,NaN,53,67,65,58,67,47,...,61,60,57,52,56,60,51,59,54,58
3,2021-05-05 00:00:00_m,53,59,NaN,54,59,58,56,59,50,...,55,51,58,53,58,54,52,54,57,48
4,2021-05-06 00:00:00_m,54,57,NaN,60,57,56,68,58,51,...,63,65,60,55,61,62,56,62,62,57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,2022-04-23 00:00:00_m,48,43,NaN,19,67,23,77,62,9,...,75,79,70,51,74,75,50,75,68,47
357,2022-04-24 00:00:00_m,45,72,9,13,73,13,59,73,8,...,53,48,56,49,54,53,51,50,47,46
358,2022-04-25 00:00:00_m,42,NaN,-3,0,57,4,37,47,2,...,38,34,35,33,34,39,30,37,30,40
359,2022-04-26 00:00:00_m,36,NaN,3,1,50,6,38,35,-1,...,43,43,36,31,36,43,38,43,35,34


In [71]:
temp_maxes = temp_maxes.reset_index(drop=True)
temp_maxes

,index,GHCND:CA006020559,GHCND:USC00210018,GHCND:USC00210050,GHCND:USC00210059,GHCND:USC00210075,GHCND:USC00210190,GHCND:USC00210287,GHCND:USC00210355,GHCND:USC00210387,...,GHCND:USW00014927,GHCND:USW00014992,GHCND:USW00054932,GHCND:USW00094931,GHCND:USW00094938,GHCND:USW00094960,GHCND:USW00094961,GHCND:USW00094963,GHCND:USW00094967,GHCND:USW00094992
0,2021-05-02 00:00:00_y,72,86,NaN,89,91,88,87,91,77,...,68,71,70,75,73,67,70,70,75,53
1,2021-05-03 00:00:00_y,48,72,NaN,66,83,65,64,82,73,...,68,64,60,53,59,68,50,67,58,50
2,2021-05-04 00:00:00_y,50,52,NaN,53,67,65,58,67,47,...,61,60,57,52,56,60,51,59,54,58
3,2021-05-05 00:00:00_y,53,59,NaN,54,59,58,56,59,50,...,55,51,58,53,58,54,52,54,57,48
4,2021-05-06 00:00:00_y,54,57,NaN,60,57,56,68,58,51,...,63,65,60,55,61,62,56,62,62,57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,2022-04-23 00:00:00_y,48,43,NaN,19,67,23,77,62,9,...,75,79,70,51,74,75,50,75,68,47
357,2022-04-24 00:00:00_y,45,72,9,13,73,13,59,73,8,...,53,48,56,49,54,53,51,50,47,46
358,2022-04-25 00:00:00_y,42,NaN,-3,0,57,4,37,47,2,...,38,34,35,33,34,39,30,37,30,40
359,2022-04-26 00:00:00_y,36,NaN,3,1,50,6,38,35,-1,...,43,43,36,31,36,43,38,43,35,34


# GDD

In [84]:
# calculate the mean temps and GDD
for i in range(1, len(list(temp_maxes.columns))):
    for j in range(len(temp_maxes.iloc[:,i])):
        temp_means.iloc[:,i][j] =(temp_maxes.iloc[:,i].astype(float)[j] + temp_mins.iloc[:,i].astype(float)[j])/2
        gdd_val = temp_means.iloc[:,i][j] - 50
        if gdd_val < 0:
            temp_gdd.iloc[:,i][j] = 0
        else:
            temp_gdd.iloc[:,i][j] = gdd_val

temp_gdd 

,index,GHCND:CA006020559,GHCND:USC00210018,GHCND:USC00210050,GHCND:USC00210059,GHCND:USC00210075,GHCND:USC00210190,GHCND:USC00210287,GHCND:USC00210355,GHCND:USC00210387,...,GHCND:USW00014927,GHCND:USW00014992,GHCND:USW00054932,GHCND:USW00094931,GHCND:USW00094938,GHCND:USW00094960,GHCND:USW00094961,GHCND:USW00094963,GHCND:USW00094967,GHCND:USW00094992
0,2021-05-02 00:00:00_g,3.5,15,NaN,13.5,24.5,18.5,20,18.5,9,...,9.5,11.5,6,9,11.5,8.5,2,12,12,0
1,2021-05-03 00:00:00_g,0,5.5,NaN,1,14.5,5,4,18.5,5.5,...,9,3.5,0,0,1,7.5,0,7.5,0,0
2,2021-05-04 00:00:00_g,0,0,NaN,0,3.5,1,0,3,0,...,1,0,0,0,0,0,0,0,0,0
3,2021-05-05 00:00:00_g,0,0,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2021-05-06 00:00:00_g,0,0,NaN,0,0,0,1.5,0,0,...,0.5,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,2022-04-23 00:00:00_g,0,0,NaN,0,10.5,0,13,4.5,0,...,14,11,5.5,0,9,14,0,12.5,4.5,0
357,2022-04-24 00:00:00_g,0,4.5,0,0,10,0,0,8,0,...,0,0,0,0,0,0,0,0,0,0
358,2022-04-25 00:00:00_g,0,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
359,2022-04-26 00:00:00_g,0,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [85]:
updated_temp_data = pd.concat([temp_mins, temp_maxes, temp_means, temp_gdd]).reset_index(drop=True)
updated_temp_data

,index,GHCND:CA006020559,GHCND:USC00210018,GHCND:USC00210050,GHCND:USC00210059,GHCND:USC00210075,GHCND:USC00210190,GHCND:USC00210287,GHCND:USC00210355,GHCND:USC00210387,...,GHCND:USW00014927,GHCND:USW00014992,GHCND:USW00054932,GHCND:USW00094931,GHCND:USW00094938,GHCND:USW00094960,GHCND:USW00094961,GHCND:USW00094963,GHCND:USW00094967,GHCND:USW00094992
0,2021-05-02 00:00:00_x,35,44,NaN,38,58,49,53,46,41,...,51,52,42,43,50,50,34,54,49,39
1,2021-05-03 00:00:00_x,43,39,NaN,36,46,45,44,55,38,...,50,43,40,38,43,47,39,48,41,40
2,2021-05-04 00:00:00_x,36,25,NaN,38,40,37,30,39,35,...,41,36,33,27,32,37,34,41,30,39
3,2021-05-05 00:00:00_x,24,24,NaN,27,37,29,35,34,25,...,35,33,28,22,28,33,25,37,28,34
4,2021-05-06 00:00:00_x,23,25,NaN,29,39,30,35,38,23,...,38,34,31,23,35,40,25,37,29,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1439,2022-04-23 00:00:00_g,0,0,NaN,0,10.5,0,13,4.5,0,...,14,11,5.5,0,9,14,0,12.5,4.5,0
1440,2022-04-24 00:00:00_g,0,4.5,0,0,10,0,0,8,0,...,0,0,0,0,0,0,0,0,0,0
1441,2022-04-25 00:00:00_g,0,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1442,2022-04-26 00:00:00_g,0,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [92]:
updated_temp_data_tp = updated_temp_data.transpose().iloc[1:].rename(columns=updated_temp_data.transpose().iloc[0]).reset_index()
updated_temp_data_tp


,index,2021-05-02 00:00:00_x,2021-05-03 00:00:00_x,2021-05-04 00:00:00_x,2021-05-05 00:00:00_x,2021-05-06 00:00:00_x,2021-05-07 00:00:00_x,2021-05-08 00:00:00_x,2021-05-09 00:00:00_x,2021-05-10 00:00:00_x,...,2022-04-18 00:00:00_g,2022-04-19 00:00:00_g,2022-04-20 00:00:00_g,2022-04-21 00:00:00_g,2022-04-22 00:00:00_g,2022-04-23 00:00:00_g,2022-04-24 00:00:00_g,2022-04-25 00:00:00_g,2022-04-26 00:00:00_g,2022-04-27 00:00:00_g
0,GHCND:CA006020559,35,43,36,24,23,28,25,28,30,...,0,0,0,0,0,0,0,0,0,0
1,GHCND:USC00210018,44,39,25,24,25,23,25,25,27,...,NaN,NaN,NaN,0,0,0,4.5,NaN,NaN,NaN
2,GHCND:USC00210050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
3,GHCND:USC00210059,38,36,38,27,29,28,30,34,30,...,0,0,0,0,0,0,0,0,0,0
4,GHCND:USC00210075,58,46,40,37,39,41,38,38,41,...,0,0,0,0,2.5,10.5,10,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,GHCND:USW00094960,50,47,37,33,40,34,33,36,33,...,0,0,0,0,0,14,0,0,0,0
174,GHCND:USW00094961,34,39,34,25,25,30,28,33,30,...,0,0,0,0,0,0,0,0,0,0
175,GHCND:USW00094963,54,48,41,37,37,38,37,37,40,...,0,0,0,0,0,12.5,0,0,0,0
176,GHCND:USW00094967,49,41,30,28,29,29,25,29,30,...,0,0,0,0,0,4.5,0,0,0,0


In [104]:
unique_stations = meteostat_data.groupby('station_id').mean().reset_index()[['station_id', 'lat', 'lon']]
unique_stations

,station_id,lat,lon
0,GHCND:CA006020559,48.633300,-93.966700
1,GHCND:USC00210050,48.300500,-95.981600
2,GHCND:USC00210059,46.525700,-93.667400
3,GHCND:USC00210075,43.606400,-93.301900
4,GHCND:USC00210190,45.253400,-93.292700
...,...,...,...
156,GHCND:USW00094960,45.062220,-93.351070
157,GHCND:USW00094961,48.726060,-94.612160
158,GHCND:USW00094963,44.832140,-93.470510
159,GHCND:USW00094967,46.899670,-95.066820


In [112]:
joined = pd.merge(unique_stations, updated_temp_data_tp, how="left", left_on="station_id", right_on = "index")
joined = joined.drop(['index'], axis=1)

joined

,station_id,lat,lon,2021-05-02 00:00:00_x,2021-05-03 00:00:00_x,2021-05-04 00:00:00_x,2021-05-05 00:00:00_x,2021-05-06 00:00:00_x,2021-05-07 00:00:00_x,2021-05-08 00:00:00_x,...,2022-04-18 00:00:00_g,2022-04-19 00:00:00_g,2022-04-20 00:00:00_g,2022-04-21 00:00:00_g,2022-04-22 00:00:00_g,2022-04-23 00:00:00_g,2022-04-24 00:00:00_g,2022-04-25 00:00:00_g,2022-04-26 00:00:00_g,2022-04-27 00:00:00_g
0,GHCND:CA006020559,48.633300,-93.966700,35,43,36,24,23,28,25,...,0,0,0,0,0,0,0,0,0,0
1,GHCND:USC00210050,48.300500,-95.981600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
2,GHCND:USC00210059,46.525700,-93.667400,38,36,38,27,29,28,30,...,0,0,0,0,0,0,0,0,0,0
3,GHCND:USC00210075,43.606400,-93.301900,58,46,40,37,39,41,38,...,0,0,0,0,2.5,10.5,10,0,0,0
4,GHCND:USC00210190,45.253400,-93.292700,49,45,37,29,30,31,28,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,GHCND:USW00094960,45.062220,-93.351070,50,47,37,33,40,34,33,...,0,0,0,0,0,14,0,0,0,0
157,GHCND:USW00094961,48.726060,-94.612160,34,39,34,25,25,30,28,...,0,0,0,0,0,0,0,0,0,0
158,GHCND:USW00094963,44.832140,-93.470510,54,48,41,37,37,38,37,...,0,0,0,0,0,12.5,0,0,0,0
159,GHCND:USW00094967,46.899670,-95.066820,49,41,30,28,29,29,25,...,0,0,0,0,0,4.5,0,0,0,0


In [113]:
joined.to_csv(r"joined_gdd_station.csv")

In [116]:
out = r'C:\Users\umn-ahmad178\Documents\ArcGIS\Projects\final_arc2\final_arc2.gdb\gdd_stations'
arcpy.management.XYTableToPoint(r"joined_gdd_station.csv", out,
                                "lon", "lat")


<Result 'C:\\Users\\umn-ahmad178\\Documents\\ArcGIS\\Projects\\final_arc2\\final_arc2.gdb\\gdd_stations'>

In [9]:
sample_days = ['21-05-02 00:00:00_g',
'21-05-03 00:00:00_g',
'21-05-04 00:00:00_g',
'21-05-05 00:00:00_g',
'21-05-06 00:00:00_g',
'21-05-07 00:00:00_g',
'21-05-08 00:00:00_g']


In [39]:
pd.read_csv(r"joined_gdd_station.csv")

,Unnamed: 0,station_id,lat,lon,2021-05-02 00:00:00_x,2021-05-03 00:00:00_x,2021-05-04 00:00:00_x,2021-05-05 00:00:00_x,2021-05-06 00:00:00_x,2021-05-07 00:00:00_x,...,2022-04-18 00:00:00_g,2022-04-19 00:00:00_g,2022-04-20 00:00:00_g,2022-04-21 00:00:00_g,2022-04-22 00:00:00_g,2022-04-23 00:00:00_g,2022-04-24 00:00:00_g,2022-04-25 00:00:00_g,2022-04-26 00:00:00_g,2022-04-27 00:00:00_g
0,0,GHCND:CA006020559,48.633300,-93.966700,35.0,43.0,36.0,24.0,23.0,28.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
1,1,GHCND:USC00210050,48.300500,-95.981600,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,0
2,2,GHCND:USC00210059,46.525700,-93.667400,38.0,36.0,38.0,27.0,29.0,28.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
3,3,GHCND:USC00210075,43.606400,-93.301900,58.0,46.0,40.0,37.0,39.0,41.0,...,0.0,0.0,0.0,0.0,2.5,10.5,10.0,0.0,0,0
4,4,GHCND:USC00210190,45.253400,-93.292700,49.0,45.0,37.0,29.0,30.0,31.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,156,GHCND:USW00094960,45.062220,-93.351070,50.0,47.0,37.0,33.0,40.0,34.0,...,0.0,0.0,0.0,0.0,0.0,14.0,0.0,0.0,0,0
157,157,GHCND:USW00094961,48.726060,-94.612160,34.0,39.0,34.0,25.0,25.0,30.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
158,158,GHCND:USW00094963,44.832140,-93.470510,54.0,48.0,41.0,37.0,37.0,38.0,...,0.0,0.0,0.0,0.0,0.0,12.5,0.0,0.0,0,0
159,159,GHCND:USW00094967,46.899670,-95.066820,49.0,41.0,30.0,28.0,29.0,29.0,...,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0,0


In [44]:
import pandas as pd
loc = r'C:\Users\umn-ahmad178\Documents\ArcGIS\Projects\final_arc2\final_arc2.gdb'
lpi = ['lpi210502', 'lpi210503', 'lpi210504', 'lpi210505', 'lpi210506', 'lpi210507', 'lpi210508']
ebk = ['ebk210502', 'ebk210503', 'ebk210504', 'ebk210505', 'ebk210506', 'ebk210507', 'ebk210508']
gpi = ['gpi210502', 'gpi210503', 'gpi210504', 'gpi210505', 'gpi210506', 'gpi210507', 'gpi210508']
mn_county_points = r'C:\Users\umn-ahmad178\Desktop\lab3\final\mn_county_points.shp'
df_tot = pd.DataFrame([])
df_dict = {}
for i in range(len(lpi)):
    
    date = '20' + lpi[i][3:5] + '-' + lpi[i][5:7] + '-' + lpi[i][7:9]

    lpishp = loc + "\\" + lpi[i] + '_cv'
    ebkshp = loc + "\\" + ebk[i] + '_cv'
    gpishp = loc + "\\" + gpi[i] + '_cv'

    lpiarr = arcpy.da.TableToNumPyArray(lpishp, ('OBJECTID', 'SHAPE@X', 'SHAPE@Y', 'Measured', 'Predicted', 'Error'))
    lpidf = pd.DataFrame(lpiarr)
    lpi_err = lpidf['Error'].mean()
    
    gpiarr = arcpy.da.TableToNumPyArray(gpishp, ('OBJECTID', 'SHAPE@X', 'SHAPE@Y', 'Measured', 'Predicted', 'Error'))
    gpidf = pd.DataFrame(gpiarr)
    gpi_err = gpidf['Error'].mean()
    
    ebkarr = arcpy.da.TableToNumPyArray(ebkshp, ('OBJECTID', 'SHAPE@X', 'SHAPE@Y', 'Measured', 'Predicted', 'Error'))
    ebkdf = pd.DataFrame(ebkarr)
    ebk_err = ebkdf['Error'].mean()
    
    errs = [lpi_err, gpi_err, ebk_err]
    
    if min(errs) == lpi_err:
        df_dict.update({date:'lpi'})
        df_tot[date+'_g'] = lpidf['Measured']
        df_tot['lat'] = lpidf['SHAPE@Y']
        df_tot['lon'] = lpidf['SHAPE@X']
    elif min(errs) == gpi_err:
        df_dict.update({date:'gpi'})
        df_tot[date+'_g'] = gpidf['Measured']
        df_tot['lat'] = gpidf['SHAPE@Y']
        df_tot['lon'] = gpidf['SHAPE@X']
    else:
        df_dict.update({date:'ebk'})
        df_tot[date+'_g'] = ebkdf['Measured']
        df_tot['lat'] = ebkdf['SHAPE@Y']
        df_tot['lon'] = ebkdf['SHAPE@X']
        
    
df_dict

{'2021-05-02': 'ebk',
 '2021-05-03': 'gpi',
 '2021-05-04': 'gpi',
 '2021-05-05': 'lpi',
 '2021-05-06': 'lpi',
 '2021-05-07': 'gpi',
 '2021-05-08': 'ebk'}

In [62]:

st_shp = r'C:\Users\umn-ahmad178\Documents\ArcGIS\Projects\final_arc2\final_arc2.gdb\gdd_stations'
mn_county = r'C:\Users\umn-ahmad178\Desktop\lab3\final2\mn_county.shp'
from arcpy.sa import *
for k, v in df_dict.items():
    print(k,v)
    if v == 'ebk':
        inPointFeatures = st_shp
        zField = k + " 00:00:00_g"
        outrast = v + '_' + k[2:4] + k[5:7] + k[8:10]
        # Execute EmpiricalBayesianKriging
        arcpy.EmpiricalBayesianKriging_ga(in_features = inPointFeatures, z_field = zField, out_raster = outrast)
        
        out_point_features = outrast + '_county'
        ExtractValuesToPoints(mn_county, outrast, out_point_features)
        
    elif v == 'gpi':
        inPointFeatures = st_shp
        zField = k + " 00:00:00_g"
        outrast = v + '_' + k[2:4] + k[5:7] + k[8:10]
        # Execute GlobalPolynomialInterpolation
        arcpy.GlobalPolynomialInterpolation(in_features = inPointFeatures, z_field = zField, out_raster = outrast)
        
        out_point_features = outrast + '_county'
        ExtractValuesToPoints(mn_county, outrast, out_point_features)
    else:
        inPointFeatures = st_shp
        zField = k + " 00:00:00_g"
        outrast = v + '_' + k[2:4] + k[5:7] + k[8:10]
        # Execute LocalPolynomialInterpolation
        arcpy.LocalPolynomialInterpolation(in_features = inPointFeatures, z_field = zField, out_raster = outrast)
        
        out_point_features = outrast + '_county'
        ExtractValuesToPoints(mn_county, outrast, out_point_features)




2021-05-02 ebk
2021-05-03 gpi
2021-05-04 gpi
2021-05-05 lpi
2021-05-06 lpi
2021-05-07 gpi
2021-05-08 ebk


In [74]:
shp_names = []
for k, v in df_dict.items():
    shp_names.append(v + '_' + k[2:4] + k[5:7] + k[8:10]+ '_county')
shp_names   

['ebk_210502_county',
 'gpi_210503_county',
 'gpi_210504_county',
 'lpi_210505_county',
 'lpi_210506_county',
 'gpi_210507_county',
 'ebk_210508_county']

In [96]:
import arcpy
import psycopg2
import arcgisscripting
gp = arcgisscripting.create()
conn = psycopg2.connect("dbname='postgres' user='postgres' host='34.72.222.158' password='postgres'")#connecting to DB
conn.autocommit = True
cur = conn.cursor()  #setting up connection cursor
cur.execute('''drop table if exists testshp''')

q = '''
CREATE TABLE public.testshp
(
    gid integer,
    county_nam character varying(80) COLLATE pg_catalog."default",
    geoid character varying(80) COLLATE pg_catalog."default",
    rastervalu double precision,
    date character varying(20),
    geom geometry(Point,4326)
    
)

TABLESPACE pg_default;
'''
cur.execute(q)
conn.commit()

    


In [97]:
arcpy.env.workspace = r"C:\Users\umn-ahmad178\Documents\ArcGIS\Projects\final_arc2\final_arc2.gdb"
for shp in shp_names:  
    date = '20' + shp[4:6] + '-' + shp[6:8] + '-' + shp[8:10] #2021-05-02
    print(date)
    for row in arcpy.da.SearchCursor(shp, ['OBJECTID','county_nam','geoid', 'RASTERVALU', 'SHAPE@']):
        wkt = row[4].WKT
        geoid = row[2]
        county_nam = row[1]
        objectid = row[0]
        gdd = row[3]
        q = f'''
    INSERT INTO testshp VALUES ('{objectid}', '{county_nam}', '{geoid}', '{gdd}', '{date}', ST_setSRID(ST_GeomFromWKB(ST_AsEWKB(ST_GeomFromText('{wkt}'))),4326)

    );
    '''
        cur.execute(q)
        conn.commit()        


2021-05-02
2021-05-03
2021-05-04
2021-05-05
2021-05-06
2021-05-07
2021-05-08
